# 01 Import libraries

In [2]:
import numpy as np
import torch


device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: cpu


In [3]:
import pandas as pd

In [4]:
import importlib

In [5]:
%reload_ext autoreload
%autoreload 2

In [6]:
import lib
importlib.reload(lib)

<module 'lib' from 'c:\\Users\\Cumberkid\\OneDrive\\Desktop\\New results\\Portfolio Allocation\\Learning-the-Optimal-Solution-Path\\lib\\__init__.py'>

In [17]:
from lib.fast_tensor_data_loader import FastTensorDataLoader
from lib.ngs.naive_grid_search_2d import naive_grid_search_2d
from lib.ngs.utils_ngs_2d import get_losses_2d
from lib.ngs.loss_fn import allocation_cost

# 02 Load data

In [18]:
# file path for Colab. May need to change this
# X_df = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/data/X_processed.csv')
# y_df = pd.read_csv('/content/Learning-the-Optimal-Solution-Path/experiments/fair-regression/data/y_processed.csv')
decomp_cov_df = pd.read_csv('decomp_cov.csv')
mean_df = pd.read_csv('mean.csv')

In [19]:
decomp_cov = np.array(decomp_cov_df)
mean = np.array(mean_df).squeeze()

In [20]:
decomp_cov = torch.tensor(decomp_cov, dtype=torch.float32)
mean = torch.tensor(mean, dtype=torch.float32)

In [21]:
# full gradient descent uses all data points
GD_data_loader = FastTensorDataLoader(decomp_cov, mean, batch_size=len(decomp_cov), shuffle=False, )
# test data
test_data_loader = FastTensorDataLoader(decomp_cov, mean, batch_size=len(decomp_cov), shuffle=False, )

# 03 Compute the true solution path

We use the Naive Grid Search with $2^{10}$ grids, trained by full gradient descent with tuned learning rate = $2$ and 5000 iterations on each grid, to generate a true solution path.

In [22]:
lam_max_2d = [1, 1]
lam_min_2d = [0, 0]
input_dim = decomp_cov.shape[1] - 1

In [23]:
loss_fn = allocation_cost

In [24]:
# num_grid = 2**10
num_grid_2d = [20, 20]
lambdas = [np.linspace(lam_max_2d[0], lam_min_2d[0], num_grid_2d[0]), np.linspace(lam_max_2d[1], lam_min_2d[1], num_grid_2d[1])]
fine_delta_lam_2d = (np.array(lam_max_2d) - np.array(lam_min_2d))/(np.array(num_grid_2d) - 1)

In [25]:
epochs = 1000
lr = .5**7 # previously tuned

In [26]:
total_itr_2d, hyper_params_2d, intercepts_2d, weights_2d, grid_pass_error_2d = naive_grid_search_2d(lam_min_2d, lam_max_2d, num_grid_2d,
                                epochs, loss_fn, GD_data_loader, input_dim, lr=lr, device=device)

In [27]:
total_itr_2d #34.5s for 50000
# 7m for 400000

400000

In [28]:
np.array(hyper_params_2d).shape

(20, 20, 2)

In [29]:
len(weights_2d)

20

In [31]:
losses = get_losses_2d(lam_min_2d, lam_max_2d, fine_delta_lam_2d, intercepts_2d,
                              weights_2d, np.array(hyper_params_2d), test_data_loader, loss_fn, device)

In [33]:
np.array(losses).shape

(20, 20)

In [36]:
thetas = np.array(weights_2d)
print(thetas.shape)

(20, 20, 9)


In [37]:
exact_soln_list = pd.DataFrame(losses)

# Save the DataFrame to a CSV file
exact_soln_list.to_csv('exact_soln_list_2d.csv', index=False)

In [38]:
# Read the CSV file into a DataFrame
truth = pd.read_csv('exact_soln_list_2d.csv')

# Display the DataFrame
truth

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,15.195165,11.914677,11.955276,11.995770,12.036159,12.076448,12.116634,12.156718,12.196705,12.236592,12.276388,12.316086,12.355691,12.395209,12.434635,12.473982,12.513460,12.559717,12.605769,12.652280
1,14.411641,11.323114,11.363797,11.404370,11.444837,11.485199,11.525454,11.565603,11.605650,11.645598,11.685446,11.725196,11.764850,11.804410,11.843878,11.883253,11.922541,11.961743,12.000860,12.039893
2,13.628118,10.730240,10.771020,10.811688,10.852239,10.892681,10.933014,10.973238,11.013354,11.053367,11.093277,11.133083,11.172795,11.212406,11.251925,11.291348,11.330680,11.369923,11.409075,11.448140
3,12.844594,10.135834,10.176719,10.217484,10.258136,10.298669,10.339087,10.379393,10.419589,10.459677,10.499657,10.539532,10.579308,10.618981,10.658557,10.698036,10.737419,10.776708,10.815906,10.855016
4,12.061070,9.539598,9.580598,9.621478,9.662234,9.702869,9.743388,9.783785,9.824072,9.864246,9.904308,9.944264,9.984114,10.023858,10.063503,10.103045,10.142489,10.181840,10.221090,10.260247
5,11.277547,8.941166,8.982296,9.023297,9.064174,9.104922,9.145550,9.186058,9.226444,9.266717,9.306875,9.346920,9.386857,9.426688,9.466413,9.506031,9.545547,9.584966,9.624282,9.663498
6,10.494023,8.340089,8.381352,8.422487,8.463491,8.504368,8.545116,8.585744,8.626248,8.666632,8.706898,8.747054,8.787091,8.827019,8.866837,8.906548,8.946154,8.985652,9.025046,9.064337
7,9.710500,7.735779,7.777187,7.818461,7.859601,7.900612,7.941496,7.982255,8.022887,8.063400,8.103793,8.144068,8.184226,8.224271,8.264201,8.304020,8.343726,8.383324,8.422812,8.462193
8,8.926976,7.127531,7.169084,7.210499,7.251784,7.292938,7.333964,7.374864,7.415640,7.456291,7.496825,7.537237,7.577532,7.617707,7.657767,7.697713,7.737544,7.777260,7.816865,7.856357
9,8.143453,6.514608,6.556315,6.597885,6.639320,6.680623,6.721797,6.762846,6.803771,6.844573,6.885253,6.925814,6.966257,7.006580,7.046786,7.086874,7.126845,7.166700,7.206441,7.246068
